This code calculates further on X_test
A selection of 100 molecules is made, based on the selection from pipeline + further reduction by similarity analysis of the selected molecules with the known ALDH1-positives (and negatives)
Can be appended to Constantijns pipeline.
Als er problemen zijn laat maar weten.

## Further processing of the selection

Create new datafame from X_test. Append values for 'ALDH1_inhibition' and 'predicted'

In [20]:
pipeline_results = X_test.copy()
pipeline_results['ALDH1_inhibition'] = y_test
pipeline_results['predicted'] = y_pred
pipeline_results.head()

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPLOW,BCUT2D_MRLOW,SMR_VSA1,SMR_VSA5,SlogP_VSA2,EState_VSA10,VSA_EState2,FractionCSP3,NumAliphaticRings,ALDH1_inhibition,predicted
1860,12.938670,0.088947,-3.642991,2.351664,-2.307988,-2.449569,-0.125741,13.212334,38.129358,41.917069,13.212334,13.750281,0.500000,1,0,1
353,4.597214,0.819103,0.819103,2.225038,-2.049995,-2.684847,-0.460778,5.101408,0.000000,16.531283,0.000000,4.518570,0.000000,1,0,0
1333,12.173495,0.014973,-0.504740,2.132749,-2.018149,-2.195873,0.097039,9.211688,13.468494,10.350345,9.589074,24.034309,0.125000,0,0,0
905,12.442508,0.089206,-0.214087,2.206277,-2.106670,-2.313198,0.086127,14.268263,25.869347,36.336766,4.794537,12.870177,0.411765,1,1,0
1289,5.116451,0.451150,0.451150,1.975855,-2.136225,-2.479488,0.195439,4.736863,6.420822,30.486310,0.000000,0.000000,0.333333,0,1,0


Select only the predicted positives

In [21]:
pipeline_positives = pipeline_results[pipeline_results['predicted'].values == 1]
print("{} positives from pipeline".format(pipeline_positives.shape[0]))

191 positives from pipeline


## Further Selection with Tanamoto similarity

Load all known molecules in frame

In [22]:
from rdkit import DataStructs
tested_molecules = dataset_molecules.copy()
PandasTools.AddMoleculeColumnToFrame(tested_molecules, smilesCol='SMILES')

Divide molecules in Inhibitors and non-Inhibitors

In [23]:
tested_molecules_pos = tested_molecules[tested_molecules['ALDH1_inhibition'] == 1]
tested_molecules_neg = tested_molecules[tested_molecules['ALDH1_inhibition'] == 0]
print("{}% inhibitors present in known molecules".format(100*len(train_set_pos)/len(tested_molecules)))

24.15% inhibitors present in known molecules


Convert both sets to Morgan Fingerprints (to compare with)

In [30]:
from rdkit.Chem import AllChem
bulk_pos = [AllChem.GetMorganFingerprintAsBitVect(x,2) for x in tested_molecules_pos['ROMol']]
bulk_neg = [AllChem.GetMorganFingerprintAsBitVect(x,2) for x in tested_molecules_neg['ROMol']]
len_pos = len(bulk_pos); len_neg = len(bulk_neg)

Get indices from pipeline prediction and select these indices from dataset_molecules.
This must be done to retreive SMILES again, since these are needed for the comparison in the for loop

In [31]:
index_pipeline_positives = pipeline_positives.index
pipeline_positives_mol = dataset_molecules.iloc[index_pipeline_positives]
print("{} positive molecules from pipeline".format(len(pipeline_positives_mol)))

191 positive molecules from pipeline


Calculate similarity with tested positives/negatives for every mol from pipeline_positives

In [32]:
pipeline_positives_mol_copy = pipeline_positives_mol.copy(deep=False)
# Loop all rows from test set
for index, row in pipeline_positives_mol.iterrows():     
    test_smiles = row['SMILES']
    # Make allchem MolFrom the SMILES and convert it into to GetMorganFingerprintAsBitVec
    test_mol = AllChem.MolFromSmiles(test_smiles)
    ref_fps = AllChem.GetMorganFingerprintAsBitVect(test_mol,2)
    # Calculate similarities with positives and negative
    similarity_pos = [DataStructs.FingerprintSimilarity(ref_fps,x) for x in bulk_pos]
    similarity_neg = [DataStructs.FingerprintSimilarity(ref_fps,x) for x in bulk_neg]
    # Add all positive and negative similarities and normalize
    rel_sum_pos = sum(similarity_pos)/len_pos # divide by sizes to normalize the result
    rel_sum_neg = sum(similarity_neg)/len_neg # divide by sizes to normalize the result
    
    pipeline_positives_mol_copy.loc[index, ['SIM_WITH_POSITIVE_SET']] = rel_sum_pos
    pipeline_positives_mol_copy.loc[index, ['SIM_WITH_NEGATIVE_SET']] = rel_sum_neg   

pipeline_positives_mol_copy.head()

,SMILES,ALDH1_inhibition,ROMol,SIM_WITH_POSITIVE_SET,SIM_WITH_NEGATIVE_SET
1860,Cc1n[nH]c(C)c1S(=O)(=O)N1CCCC(C(=O)NCc2cccs2)C1,0,<rdkit.Chem.rdchem.Mol object at 0x000001DE619...,0.147624,0.116059
938,COC(=O)c1c(-c2ccco2)csc1NC(=O)C1CC=CCC1C(=O)[O-],1,<rdkit.Chem.rdchem.Mol object at 0x000001DE619...,0.134356,0.092620
65,FC(F)(F)c1ccccc1-c1cc(NCc2cccs2)ncn1,0,<rdkit.Chem.rdchem.Mol object at 0x000001DE609...,0.086200,0.118449
56,N#CC(Cc1ccccc1OC(F)F)c1nc2ccccc2[nH]1,0,<rdkit.Chem.rdchem.Mol object at 0x000001DE609...,0.075265,0.106971
746,Cc1onc(-c2c(F)cccc2Cl)c1C(=O)NC(=S)NC1CCSC1=O,1,<rdkit.Chem.rdchem.Mol object at 0x000001DE609...,0.135623,0.112198


In [34]:
false_positives = pipeline_positives_mol_copy[pipeline_positives_mol_copy['ALDH1_inhibition'] == 0]
print("{:d} predicted positives from pipeline. {:.1f}% false. ".format(len(pipeline_positives_mol_copy),100*len(false_positives)/len(pipeline_positives_mol_copy)))

191 predicted positives from pipeline. 47.6% false. 


Only select molecules with larger similarity with positives than with negatives

In [35]:
selection = pipeline_positives_mol_copy[pipeline_positives_mol_copy['SIM_WITH_POSITIVE_SET'] > pipeline_positives_mol_copy['SIM_WITH_NEGATIVE_SET']]
false_positives = selection[selection['ALDH1_inhibition']==0]
print("{:d} predicted positives after first selection. {:.1f}% false positive. ".format(len(selection),100*len(false_positives)/len(selection)))

172 predicted positives after first selection. 44.2% false positive. 


Sort the selection by similarity with positives and pick the first 100 molecules

In [36]:
selection_copy = selection.copy()
selection_copy = selection_copy.sort_values(by=['SIM_WITH_POSITIVE_SET'], ascending=False);
selection_copy = selection_copy.iloc[0:100]
false_positives = selection_copy[selection_copy['ALDH1_inhibition']==0]
print("{:d} predicted positives after second selection. {:.1f}% false positive. ".format(len(selection_copy),100*len(false_positives)/len(selection_copy)))

100 predicted positives after second selection. 38.0% false positive. 


Try if low score on similarity with negatives works better...

In [37]:
selection_copy = selection.copy()
selection_copy = selection_copy.sort_values(by=['SIM_WITH_NEGATIVE_SET'], ascending=True);
selection_copy = selection_copy.iloc[0:100]
false_positives = selection_copy[selection_copy['ALDH1_inhibition']==0]
print("{:d} predicted positives after second selection. {:.1f}% false positive. ".format(len(selection_copy),100*len(false_positives)/len(selection_copy)))

100 predicted positives after second selection. 39.0% false positive. 
